In [ ]:
# Import dependencies
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
import os
import pandas as pd
import string


In [ ]:
# Download NLTK resources (if not already downloaded)
nltk.download('punkt')


In [ ]:
# Creat a function to extract the words if the html file contains a body tag
def extract_words_from_body(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

        # Parse HTML using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html5lib')

        # Check if <body> tag exists in the HTML content
        body_content = soup.body
        if body_content:
            # Extract text from the body tag
            body_text = body_content.get_text()
        else:
            # Skip processing for files without <body> tag
            print(f"No <body> tag found in {file_path}. Skipping...")
            return None

        # Tokenize words using NLTK and filter out punctuation
        words = word_tokenize(body_text)
        words = [word for word in words if word.isalpha()]  # Exclude non-alphabetic tokens

        return words


In [ ]:
# Replace 'path_to_html_files_directory' with the path to your HTML files directory
directory = 'emails'


In [ ]:
# Create an empty dictionary to store word counts for each file
word_count_dict = {}


In [ ]:
# Loop through HTML files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.html'):
        file_path = os.path.join(directory, filename)

        # Extract words from the body of each HTML file and remove punctuation
        words = extract_words_from_body(file_path)

        # If words is None (indicating no <body> tag), skip to the next file
        if words is None:
            continue

        # Count word occurrences for the current email
        word_count = {}
        for word in set(words):  # Use set to get unique words per email
            word_count[word] = words.count(word)

        # Store word counts for the current email in word_count_dict
        word_count_dict[filename] = word_count



In [ ]:
# Create a DataFrame from word_count_dict
df = pd.DataFrame.from_dict(word_count_dict, orient='index')
df.fillna(0, inplace=True)  # Fill NaN values with 0



In [ ]:
# Add a 'Spam Indicator' column filled with zeros
df['Spam Indicator'] = 0


In [ ]:
# Change 'Spam Indicator' column value to 1 if the filename starts with 'spam'
df.loc[df.index.str.endswith('spam.html'), 'Spam Indicator'] = 1

In [ ]:
# Filter words that are less than or equal to 20 characters
filtered_words = [word for word in df.columns[1:-1] if len(word) <= 20]

# Summing up the word counts across all emails for filtered words
word_counts_total = df[filtered_words].sum(axis=0)

# Sort the words based on their total counts in descending order and select the top 5000
top_5000_words = word_counts_total.nlargest(5000).index

# Filter the DataFrame to keep only the top 5000 columns and include the first and last columns
df_top_5000 = df.iloc[:, [0] + [df.columns.get_loc(col) for col in top_5000_words] + [-1]]

In [ ]:
# Convert columns to integer type
df_top_5000 = df_top_5000.astype(int)

In [ ]:
df_top_5000.columns.unique()

In [ ]:
# Display the DataFrame
df_top_5000.head()

In [ ]:
# Save the DataFrame to a CSV file in the 'outputs' directory
output_directory = 'outputs'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

output_file_path = os.path.join(output_directory, 'emails_df.csv')
df_top_5000.to_csv(output_file_path)

print(f"DataFrame saved to {output_file_path}")